In [1]:
import requests 
import pandas as pd
from selenium import webdriver

from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By

In [2]:
df = pd.read_csv('planilha.csv')
# PANORÂMA INICIAL DO DATAFARME

print(df.info())
df

# ANÁLISE 1: TODOS OS DADOS ESTÃO COMO STRING

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1800 entries, 0 to 1799
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Titulo        1800 non-null   object
 1   Área          1800 non-null   int64 
 2   Quartos       1800 non-null   object
 3   Banheiros     1800 non-null   object
 4   Vaga Garagem  1800 non-null   object
 5   Valor         1800 non-null   object
 6   Endereço      1800 non-null   object
dtypes: int64(1), object(6)
memory usage: 98.6+ KB
None


,Titulo,Área,Quartos,Banheiros,Vaga Garagem,Valor,Endereço
0,"Casa com 2 Quartos à Venda, 78m²",78,2,2,2,R$ 290.000,"Roma, Volta Redonda - RJ"
1,"Apartamento com 3 Quartos à Venda, 96m²",96,3,3,3,R$ 299.900 Preço abaixo do mercado,"Monte Castelo, Volta Redonda - RJ"
2,"Apartamento com 3 Quartos à Venda, 110m²",110,3,3,3,R$ 590.000,"Aterrado, Volta Redonda - RJ"
3,"Apartamento com 2 Quartos à Venda, 50m²",50,2,2,2,R$ 240.000,"Estrada da Companhia, 5 - Roma, Volta Redonda ..."
4,"Casa com 2 Quartos à Venda, 60m²",60,2,2,2,R$ 190.000,"Avenida Pavão - Parque das Garcas, Volta Redon..."
...,...,...,...,...,...,...,...
1795,"Lote/Terreno à Venda, 421m²",421,--,--,--,R$ 294.770,"Rua 14 - Casa de Pedra, Volta Redonda - RJ"
1796,"Lote/Terreno à Venda, 268m²",268,--,--,--,R$ 190.000,"Rodovia dos Metalúrgicos - Casa de Pedra, Volt..."
1797,"Lote/Terreno à Venda, 2500m²",2500,--,--,--,R$ 150.000,"Rua Agmar Lópes Correa - Santa Cruz, Volta Red..."
1798,"Apartamento com 2 Quartos à Venda, 60m²",60,2,2,2,R$ 175.000,Avenida Major Aníbal de Oliveira Filho - Santa...


In [3]:
# ANÁLISE ÁREA 1: PERCEBESSE QUE EXISTEM VALORES PREENCHIDOS POR ' -- ', NÃO ESTAMOS INTERESSADOS EM IMÓVEIS SEM ÁREA E NÃO TEM COMO CORRIGIR 
df['Área'].value_counts()

Área
70      150
110     100
50      100
60      100
42      100
96      100
78       50
430      50
58       50
46       50
421      50
253      50
514      50
240      50
268      50
2500     50
120      50
255      50
335      50
420      50
86       50
62       50
129      50
101      50
104      50
56       50
441      50
45       50
48       50
Name: count, dtype: int64

In [4]:
# TRATAMENTO DE DADOS 1: ELIMINANDO IMÓVEIS SEM ÁREA
dp_areas = df.loc[lambda df: df['Área'] == ' -- '].index

new_df = df
for i in dp_areas:
    new_df = new_df.drop(i)

new_df

,Titulo,Área,Quartos,Banheiros,Vaga Garagem,Valor,Endereço
0,"Casa com 2 Quartos à Venda, 78m²",78,2,2,2,R$ 290.000,"Roma, Volta Redonda - RJ"
1,"Apartamento com 3 Quartos à Venda, 96m²",96,3,3,3,R$ 299.900 Preço abaixo do mercado,"Monte Castelo, Volta Redonda - RJ"
2,"Apartamento com 3 Quartos à Venda, 110m²",110,3,3,3,R$ 590.000,"Aterrado, Volta Redonda - RJ"
3,"Apartamento com 2 Quartos à Venda, 50m²",50,2,2,2,R$ 240.000,"Estrada da Companhia, 5 - Roma, Volta Redonda ..."
4,"Casa com 2 Quartos à Venda, 60m²",60,2,2,2,R$ 190.000,"Avenida Pavão - Parque das Garcas, Volta Redon..."
...,...,...,...,...,...,...,...
1795,"Lote/Terreno à Venda, 421m²",421,--,--,--,R$ 294.770,"Rua 14 - Casa de Pedra, Volta Redonda - RJ"
1796,"Lote/Terreno à Venda, 268m²",268,--,--,--,R$ 190.000,"Rodovia dos Metalúrgicos - Casa de Pedra, Volt..."
1797,"Lote/Terreno à Venda, 2500m²",2500,--,--,--,R$ 150.000,"Rua Agmar Lópes Correa - Santa Cruz, Volta Red..."
1798,"Apartamento com 2 Quartos à Venda, 60m²",60,2,2,2,R$ 175.000,Avenida Major Aníbal de Oliveira Filho - Santa...


In [5]:
# ANÁLISE QUARTOS 1: PERCEBESSE QUE EXISTEM VALORES PREENCHIDOS POR ' -- ', A QUANTIDADE DE QUARTOS PODE SER 0 OU ATÉ MESMO 1, DEVEMOS VERIFICAR SE OS IMÓVEIS TRATAM-SE DE LOTES OU RESIDÊNCIAS
df['Área'].value_counts()

Área
70      150
110     100
50      100
60      100
42      100
96      100
78       50
430      50
58       50
46       50
421      50
253      50
514      50
240      50
268      50
2500     50
120      50
255      50
335      50
420      50
86       50
62       50
129      50
101      50
104      50
56       50
441      50
45       50
48       50
Name: count, dtype: int64

In [6]:
# TRATAMENTO DE DADOS 2: VERIFICAÇÃO DOS IMÓVEIS COM QUARTOS ' -- ' -> LOTES OU RESIDÊNCIAS
# PARA LOTES -> QUARTOS = 0
# PARA RESIDÊNCIAS -> QUARTOS = 1

lotes_index = df.loc[df['Quartos'] == ' -- '].index

for i in lotes_index:

    df.loc[i, 'Quartos'] = 0

df['Quartos'].value_counts()

Quartos
 2     850
0      600
 3     300
 4      50
Name: count, dtype: int64

In [7]:
# 

# VEMOS QUE TODOS AS LINHAS ONDE (QUATROS == ' -- ') SE TRATA De UMA SALA OU SALAS COMERCIAIS, PORTANTO, DEVEMOS SUBSTITUIR VALORES POR 0
salas_index = df.loc[df['Titulo'] == '  Sala/Conjunto    à Venda, 70m²  '].index

for i in salas_index:

    df.loc[i, 'Quartos'] = 0

df['Quartos'].value_counts()

Quartos
 2     850
0      600
 3     300
 4      50
Name: count, dtype: int64

In [8]:
# VEMOS QUE TODOS AS LINHAS ONDE (QUATROS == ' -- ') SE TRATA DE LOTES OU SALAS COMERCIAIS, PORTANTO, DEVEMOS SUBSTITUIR VALORES POR 0
lotes_index = df.loc[df['Quartos'] == ' -- '].index

for i in lotes_index:

    df.loc[i, 'Quartos'] = 0

df['Quartos'].value_counts()

Quartos
 2     850
0      600
 3     300
 4      50
Name: count, dtype: int64

In [9]:
# TRATAMENTO DE DADOS 2: VERIFICAÇÃO DOS IMÓVEIS COM QUARTOS ' -- ' -> LOTES OU RESIDÊNCIAS
# PARA LOTES -> QUARTOS = 0
# PARA SALA COMERCIAL -> QUARTOS = 1
df_quartos = df.loc[lambda df: df['Banheiros'] == ' -- ']
print(df_quartos['Titulo'].values_count())

AttributeError: 'Series' object has no attribute 'values_count'

In [9]:
# TRATAMENTO DE DADOS 2: REMOÇÃO DA NOTAÇÃO REAL BRASILEIRO (R$)
df['Valor'] = df['Valor'].apply(lambda x: x.replace('R$', ''))
df

,Titulo,Área,Quartos,Banheiros,Vaga Garagem,Valor,Endereço
0,"Casa com 2 Quartos à Venda, 78m²",78,2,2,2,290.000,"Roma, Volta Redonda - RJ"
1,"Apartamento com 3 Quartos à Venda, 96m²",96,3,3,3,299.900 Preço abaixo do mercado,"Monte Castelo, Volta Redonda - RJ"
2,"Apartamento com 3 Quartos à Venda, 110m²",110,3,3,3,590.000,"Aterrado, Volta Redonda - RJ"
3,"Apartamento com 2 Quartos à Venda, 50m²",50,2,2,2,240.000,"Estrada da Companhia, 5 - Roma, Volta Redonda ..."
4,"Casa com 2 Quartos à Venda, 60m²",60,2,2,2,190.000,"Avenida Pavão - Parque das Garcas, Volta Redon..."
...,...,...,...,...,...,...,...
1795,"Lote/Terreno à Venda, 421m²",421,--,--,--,294.770,"Rua 14 - Casa de Pedra, Volta Redonda - RJ"
1796,"Lote/Terreno à Venda, 268m²",268,--,--,--,190.000,"Rodovia dos Metalúrgicos - Casa de Pedra, Volt..."
1797,"Lote/Terreno à Venda, 2500m²",2500,--,--,--,150.000,"Rua Agmar Lópes Correa - Santa Cruz, Volta Red..."
1798,"Apartamento com 2 Quartos à Venda, 60m²",60,2,2,2,175.000,Avenida Major Aníbal de Oliveira Filho - Santa...
